In [ ]:
import glob
import os
import ipywidgets as widgets
from IPython.display import display
import requests
from bs4 import BeautifulSoup

In [ ]:
# --- Model/scenario/observation dropdowns ---
#selected_regions = ["13"]
model_to_scenarios = {}

# Build model_to_scenarios from OGGM files
for region_code in selected_regions:
    base_url = f"https://cluster.klima.uni-bremen.de/~oggm/oggm-standard-projections/oggm_v16/2023.3/CMIP6/2100/RGI{region_code}/"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, "html.parser")
    file_links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.nc')]
    for fname in file_links:
        parts = fname.split("_")
        if len(parts) >= 7:
            model = parts[5]
            scenario = parts[6]
            model_to_scenarios.setdefault(model, set()).add(scenario)

for model in model_to_scenarios:
    model_to_scenarios[model] = sorted(model_to_scenarios[model])

model_options = sorted(model_to_scenarios.keys()) + ['Other']
model_dropdown = widgets.Dropdown(options=model_options, description='Model:')
scenario_dropdown = widgets.Dropdown(description='Scenario:')
observation_options = [
    'Mass Change (Hugonnet, 2021, 5-year period)',
    'Mass Change (Dussaillant, 2025, annual)',
    'Elevation Change (Hugonnet, 2021, 5-year period)',
    'Reference Glaciers'
]
observation_dropdown = widgets.Dropdown(options=observation_options, description='Observation:')

custom_model_text = widgets.Text(
    description='Custom Model:',
    placeholder='Enter custom model name',
    style={'description_width': 'initial'}
)

custom_scenario_text = widgets.Text(
    description='Custom Scenario:',
    placeholder='Enter custom scenario name',
    style={'description_width': 'initial'}
)

custom_model_text.layout.display = 'none'
custom_scenario_text.layout.display = 'none'   

# --- Custom source widgets (initially hidden) ---
custom_source_toggle = widgets.ToggleButtons(
    options=['Upload', 'Link'],
    description='Custom Source:',
    style={'description_width': 'initial'}
)
file_upload = widgets.FileUpload(accept='.csv', multiple=True)
link_input = widgets.Text(description="Data URL:", placeholder="Enter folder URL")

custom_source_toggle.layout.display = 'none'
file_upload.layout.display = 'none'
link_input.layout.display = 'none'

def on_model_change(change):
    if change['new'] == 'Other':
        # Hide scenario dropdown, show custom model/scenario text
        scenario_dropdown.layout.display = 'none'
        custom_model_text.layout.display = 'block'
        custom_scenario_text.layout.display = 'block'
        # Show custom source toggle and its children as appropriate
        custom_source_toggle.layout.display = 'block'
        if custom_source_toggle.value == 'Upload':
            file_upload.layout.display = 'block'
            link_input.layout.display = 'none'
        elif custom_source_toggle.value == 'Link':
            file_upload.layout.display = 'none'
            link_input.layout.display = 'block'
        else:
            file_upload.layout.display = 'none'
            link_input.layout.display = 'none'
    else:
        # Show scenario dropdown, hide custom model/scenario text
        scenarios = model_to_scenarios[model_dropdown.value]
        scenario_dropdown.options = scenarios
        scenario_dropdown.value = scenarios[0]
        scenario_dropdown.layout.display = 'block'
        custom_model_text.layout.display = 'none'
        custom_scenario_text.layout.display = 'none'
        # Hide custom source widgets
        custom_source_toggle.layout.display = 'none'
        file_upload.layout.display = 'none'
        link_input.layout.display = 'none'

def on_custom_source_change(change):
    if model_dropdown.value == 'Other':
        if change['new'] == 'Upload':
            file_upload.layout.display = 'block'
            link_input.layout.display = 'none'
        elif change['new'] == 'Link':
            file_upload.layout.display = 'none'
            link_input.layout.display = 'block'
        else:
            file_upload.layout.display = 'none'
            link_input.layout.display = 'none'

model_dropdown.observe(on_model_change, names='value')
custom_source_toggle.observe(on_custom_source_change, names='value')

# Initialize UI state
on_model_change({'new': model_dropdown.value})

display(
    model_dropdown,
    scenario_dropdown,
    observation_dropdown,
    custom_model_text,
    custom_scenario_text,
    custom_source_toggle,
    file_upload,
    link_input
)

Dropdown(description='Model:', options=('ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', …

Dropdown(description='Scenario:', layout=Layout(display='block'), options=('ssp126', 'ssp585'), value='ssp126'…

Dropdown(description='Observation:', options=('Mass Change (Hugonnet, 2021, 5-year period)', 'Mass Change (Dus…

Text(value='', description='Custom Model:', layout=Layout(display='none'), placeholder='Enter custom model nam…

Text(value='', description='Custom Scenario:', layout=Layout(display='none'), placeholder='Enter custom scenar…

ToggleButtons(description='Custom Source:', layout=Layout(display='none'), options=('Upload', 'Link'), style=T…

FileUpload(value=(), accept='.csv', description='Upload', layout=Layout(display='none'))

Text(value='', description='Data Path/URL:', layout=Layout(display='none'), placeholder='Enter folder path or …